## Decorators in the Standard Library
Python has three built-in functions that are designed to decorate methods: `property`, `classmethod` and `staticmethod`. 

Another frequently see decorator is `functools.wraps`, a helper for building well-behaved decorators. Two of the most interesting decorators in the standard library are `lru_cache` and the brand-new `singledispatch`.

### Memoization with functools.lru_cache
It implements memoization: an optimization technique that works by saving the results of previous invocations of an expensive function, avoiding repeat computations on previously used arguments. The letters LRU stand for Least Recently Used, meaning that the growth of the cache is limited by discarding the entries that have not been read for a while.

A good demonstration is to apply `lru_cache` to the painfully slow recursive function to generate the $n$-th number in the FIbonacci sequence.

In [1]:
from clockdeco import clock

@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

In [2]:
fibonacci(6)

[0.00000042s] fibonacci(0) -> 0
[0.00000121s] fibonacci(1) -> 1
[0.00039242s] fibonacci(2) -> 1
[0.00000035s] fibonacci(1) -> 1
[0.00000052s] fibonacci(0) -> 0
[0.00000116s] fibonacci(1) -> 1
[0.00016269s] fibonacci(2) -> 1
[0.00020148s] fibonacci(3) -> 2
[0.00063627s] fibonacci(4) -> 3
[0.00000062s] fibonacci(1) -> 1
[0.00000069s] fibonacci(0) -> 0
[0.00000089s] fibonacci(1) -> 1
[0.00010580s] fibonacci(2) -> 1
[0.00024145s] fibonacci(3) -> 2
[0.00000046s] fibonacci(0) -> 0
[0.00000063s] fibonacci(1) -> 1
[0.00005594s] fibonacci(2) -> 1
[0.00000048s] fibonacci(1) -> 1
[0.00000050s] fibonacci(0) -> 0
[0.00000067s] fibonacci(1) -> 1
[0.00005594s] fibonacci(2) -> 1
[0.00011114s] fibonacci(3) -> 2
[0.00022234s] fibonacci(4) -> 3
[0.00052880s] fibonacci(5) -> 5
[0.00150109s] fibonacci(6) -> 8


8

In [5]:
%%writefile fibo_demo_lru.py
import functools

from clockdeco import clock

@functools.lru_cache()
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

if __name__ == '__main__':
    print(fibonacci(6))

Writing fibo_demo_lru.py


In [6]:
print(fibonacci(30))

[0.00000292s] fibonacci(7) -> 13
[0.00026382s] fibonacci(8) -> 21
[0.00000291s] fibonacci(9) -> 34
[0.00047479s] fibonacci(10) -> 55
[0.00000242s] fibonacci(11) -> 89
[0.00065550s] fibonacci(12) -> 144
[0.00000365s] fibonacci(13) -> 233
[0.00083090s] fibonacci(14) -> 377
[0.00000333s] fibonacci(15) -> 610
[0.00100278s] fibonacci(16) -> 987
[0.00000379s] fibonacci(17) -> 1597
[0.00117381s] fibonacci(18) -> 2584
[0.00000408s] fibonacci(19) -> 4181
[0.00134663s] fibonacci(20) -> 6765
[0.00000357s] fibonacci(21) -> 10946
[0.00152607s] fibonacci(22) -> 17711
[0.00000344s] fibonacci(23) -> 28657
[0.00233123s] fibonacci(24) -> 46368
[0.00000442s] fibonacci(25) -> 75025
[0.00266338s] fibonacci(26) -> 121393
[0.00000290s] fibonacci(27) -> 196418
[0.00283080s] fibonacci(28) -> 317811
[0.00000203s] fibonacci(29) -> 514229
[0.00297231s] fibonacci(30) -> 832040
832040


It's important to note that `lru_cache` can be tuned by passing two optional arguments. It's full signature is:
```python
functools.lru_cache(maxsize=128, typed=False)
```
The `maxsize` argument determines how many call results are stored. After the cache is full, older results are discarded to make room. For optimal performance, `maxsize` should be a power of 2. The `typed` argument, if set to `True`, stores results of different argument types separately, i.e. distinguishing between float and integer arguments that are normally considered equal, like 1 and 1.0. By the way, because `lru_cache` uses a `dict` to store the results, and the keys are made from the positional and keyword arguments used in the calls, all the arguments taken by the decorated function must be *hashable*.

## Generic Functions with Single Dispatch
Imagine we are creating a tool to debug web applications. We want to be able to generate HTML displays for different types of Python objects.

We could start with a function like this:
```python
import html

def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)
```
That will work for any Python type, but now we want to extend it to generate custom display for some types:

* `str`: replace embedded newline characters with `<br>\n` and use `<p>` tags instead of `<pre>`
* `int`: show the number in decimal and hexadecimal
* `list`: output an HTML list, formatting each item according to its type.

The new `functools.singledispatch` decorator allows each module to contribute to the overall solution, and lets you easily provide a specialized function even for classes you can't edit. If you decorate a plain function with `@singledispatch`, it becomes a *generic function*: a group of functions to perform the same operation in different ways, depending on the type of the first argument.

In [14]:
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{}</p>'.format(content)

@htmlize.register(numbers.Integral)
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

In [8]:
htmlize(abs)

'<pre>&lt;built-in function abs&gt;</pre>'

In [16]:
htmlize((1, 2, 3))

'<ul>\n<li><pre>1 (0x1)</pre></li>\n<li><pre>2 (0x2)</pre></li>\n<li><pre>3 (0x3)</pre></li>\n</ul>'